In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import re
from slackclient import SlackClient
import datetime
import pandas as pd
import matplotlib

slack_api_key = pd.read_csv('access.csv')
slack_api_key = slack_api_key.set_index("name", drop = False)

In [ ]:
slack_key = slack_api_key.loc['slack',]['key']

In [ ]:
sc = SlackClient(slack_key)
def getUsers(sc):
    users = sc.api_call("users.list")
    user_df = pd.DataFrame()
    for user in users['members']:
        profile = user['profile']
        name = profile['real_name']
        user_id = user['id']
        df = pd.DataFrame([(profile['real_name'],user['id'])])
        user_df = user_df.append(df)
    user_df.columns = ['Name', 'userId']
    return(user_df)

def getChannels(sc):
    channels = sc.api_call("conversations.list",limit=1000)
    chanels['channels'][1]
    lst = []
    j=0
    for ch in chanels['channels']:
        created_at = (datetime.datetime.fromtimestamp(int(ch['created'])).strftime('%Y-%m-%d %H:%M:%S'))
        row = [j,created_at,ch['creator'],ch['name'],ch['previous_names'],ch['id']]
        lst.append(row)
        j=j+1
    channels_df = pd.DataFrame(lst)
    channels_df.columns = ['index','date_created','channel_creator_id','channel_name','channel_comment','channel_id']
    return(channels_df)

def getChannelMessages(sc,channel_name):
    channels_df = getChannels(sc)
    channels_df = channels_df.set_index("channel_name", drop = False)
    channel_id = channels_df.loc[channel_name,]['channel_id']
    data_interactions = pd.DataFrame()
    messages = pd.DataFrame()
    next_cursor = 1
    j=0
    convo = sc.api_call("conversations.history",channel=channel_id,limit=1000)
    while(next_cursor):
        next_cursor=0
        for message in convo['messages']:
            try:
                text_message = message['text']
                created_at = (datetime.datetime.fromtimestamp(int(float(message['ts']))).strftime('%Y-%m-%d %H:%M:%S'))
                df_message = pd.DataFrame([(message['ts'],created_at,message['user'],text_message)])
                messages = messages.append(df_message)
                for v in (re.findall(r'(<@\w+>)', text_message)):
                    v=re.sub("<@","",v)
                    v=re.sub(">","",v)
                    df = pd.DataFrame([(message['ts'],created_at,message['user'],v,text_message)])
                    data_interactions = data_interactions.append(df)
                    #print(j,'timestamp:',created_at,'sender:',message['user'],',',v)
                j=j+1
            except:
                print('error')
                #print(df_message)
        try:
            next_cursor = convo['response_metadata']['next_cursor']
            convo=sc.api_call("conversations.history",channel=channel_id,cursor=next_cursor,limit=1000)
            print(next_cursor)
        except:
            print('No next cursor')
            next_cursor=0
    messages.columns = ['time_unix','time','userId','message']
    messages['datetime'] = pd.to_datetime(messages['time'])
    messages = messages.set_index(['datetime'])
    data_interactions.columns = ['time_unix','time','userId_src','userId_dst','message']
    return(data_interactions,messages)



In [ ]:
user_df = getUsers(sc)
channels_df = getChannels(sc)
(interactions,messages) = getChannelMessages(sc,'watercooler')

In [ ]:
interactions.sort_values('time_unix',ascending=False)

In [ ]:
messages_labelled = pd.merge(messages,user_df,on='userId')
interactions_labelled = pd.merge(interactions,user_df,left_on='userId_src',right_on='userId',indicator=True).merge(user_df,left_on='userId_dst',right_on='userId')

In [ ]:
messages_labelled.to_csv('watercooler_messages.csv')
interactions_labelled.to_csv('watercooler_interactions.csv')

In [ ]:
messages.groupby(messages.index.dayofweek).count().plot()
df1 = messages.groupby(messages.index.dayofweek).size()